In [1]:
import torch
import time

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import Parameter

from syft.frameworks.torch.nn import rnn
from syft.federated.floptimizer import Optims
from syft.frameworks.torch.fl import utils

from functools import wraps

from torch.utils.data import Dataset, DataLoader

from torch.autograd import Variable

from torch.optim import lr_scheduler

import syft as sy

import urllib
from pathlib import Path

import copy
import math
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import MinMaxScaler, RobustScaler

from torch.utils.data import Dataset, DataLoader

from torch.autograd import Variable
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
torch.set_default_tensor_type(torch.cuda.FloatTensor)

/home/minsoo/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/minsoo/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/minsoo/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/minsoo/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [2]:
class Arguments():
    def __init__(self):
        self.batch_size = 32
        self.test_batch_size = 16
        self.seed = 1
        self.epochs = 50
        self.learning_rate = 1e-3
        self.worker_epochs = 1
        self.federate_after_n_batches = 1
        # hyper parameters
        self.hidden_dim = 64
        self.output_dim = 24

        self.look_ahead = 1*24 # a day-ahead forecasting
        self.look_back = 7*24 # using a week load data

In [3]:
args = Arguments()

In [4]:
# make dataset to input
def build_dataset(time_series, look_back, look_ahead):
    dataX = []
    dataY = []
    print(len(time_series) - look_back)
    for i in range(0, len(time_series) - look_back, look_ahead):
        sample_X = time_series[i : (i + look_back), :]
        sample_Y = time_series[i + look_back : i + look_back + look_ahead, :]
        dataX.append(sample_X)
        dataY.append(sample_Y)
        
    #print(np.asarray(dataX).shape)
    #print(np.asarray(dataY).shape)
    data_numX = np.array(dataX, dtype = 'float32')
    data_numY = np.array(dataY, dtype = 'float32')
    return data_numX, data_numY

In [5]:
class Time_Series_Data(Dataset):

    def __init__(self, train_x, train_y):
        self.X = train_x
        self.y = train_y

    def __getitem__(self, item):
        x_t = self.X[item]
        y_t = self.y[item]
        return x_t, y_t

    def __len__(self):

        return len(self.X)

In [6]:
df = pd.read_csv("weather_from_14y_8_1_to_16y.csv", index_col = 0)

scaler_temp = RobustScaler()
scaler_temp.fit(df['Temperature?'].values.reshape(-1,1))

scaler_ws = RobustScaler()
scaler_ws.fit(df['Wind_speed(m/s)'].values.reshape(-1,1))

scaler_wd = RobustScaler()
scaler_wd.fit(df['Wind_direction'].values.reshape(-1,1))

scaler_hum = RobustScaler()
scaler_hum.fit(df['Humidity(%)'].values.reshape(-1,1))

scaler_vp = RobustScaler()
scaler_vp.fit(df['Vapor_pressure(hPa)'].values.reshape(-1,1))

scaler_lp = RobustScaler()
scaler_lp.fit(df['Local_pressure(hPa)'].values.reshape(-1,1))

scaler_sr = RobustScaler()
scaler_sr.fit(df['Solar_radiation(MJ/m2)'].values.reshape(-1,1))

scaler_month = RobustScaler()
scaler_month.fit(df['month'].values.reshape(-1,1))

scaler_day = RobustScaler()
scaler_day.fit(df['day'].values.reshape(-1,1))

scaler_hour = RobustScaler()
scaler_hour.fit(df['hour'].values.reshape(-1,1))

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
             with_scaling=True)

In [7]:
def data_preprocess(dataframe, train_start, train_end, valid_start, valid_end, test_start, test_end):
    data_train = dataframe[train_start:train_end].values.reshape(-1,1)
    data_valid = dataframe[valid_start:valid_end].values.reshape(-1,1)
    data_test = dataframe[test_start:test_end].values.reshape(-1,1)
    return data_train, data_valid, data_test

In [8]:
#train_start ="2014-08-01 8:00"
#train_end = "2016-01-30 7:00"

#train_start ="2014-08-01 8:00"
#train_end = "2016-08-31 7:00"

train_start ="2014-08-01 8:00"
train_end = "2016-10-30 7:00"

valid_start = "2016-11-23 8:00"
valid_end = "2016-12-31 7:00"

test_start = "2016-11-23 8:00"
test_end = "2016-12-31 7:00"



temp_train, temp_valid, temp_test = data_preprocess(df['Temperature?'], train_start, train_end, valid_start, valid_end, test_start, test_end)

ws_train, ws_valid, ws_test = data_preprocess(df['Wind_speed(m/s)'], train_start, train_end, valid_start, valid_end, test_start, test_end)

wd_train, wd_valid, wd_test = data_preprocess(df['Wind_direction'], train_start, train_end, valid_start, valid_end, test_start, test_end)

hum_train, hum_valid, hum_test = data_preprocess(df['Humidity(%)'], train_start, train_end, valid_start, valid_end, test_start, test_end)

vp_train, vp_valid, vp_test = data_preprocess(df['Vapor_pressure(hPa)'], train_start, train_end, valid_start, valid_end, test_start, test_end)

lp_train, lp_valid, lp_test = data_preprocess(df['Local_pressure(hPa)'], train_start, train_end, valid_start, valid_end, test_start, test_end)

sr_train, sr_valid, sr_test = data_preprocess(df['Solar_radiation(MJ/m2)'], train_start, train_end, valid_start, valid_end, test_start, test_end)

month_train, month_valid, month_test = data_preprocess(df['month'], train_start, train_end, valid_start, valid_end, test_start, test_end)

day_train, day_valid, day_test = data_preprocess(df['day'], train_start, train_end, valid_start, valid_end, test_start, test_end)

hour_train, hour_valid, hour_test = data_preprocess(df['hour'], train_start, train_end, valid_start, valid_end, test_start, test_end)

temp_train_scaled = scaler_temp.transform(temp_train)
temp_valid_scaled = scaler_temp.transform(temp_valid)
temp_test_scaled = scaler_temp.transform(temp_test)

ws_train_scaled = scaler_ws.transform(ws_train)
ws_valid_scaled = scaler_ws.transform(ws_valid)
ws_test_scaled = scaler_ws.transform(ws_test)

wd_train_scaled = scaler_wd.transform(wd_train)
wd_valid_scaled = scaler_wd.transform(wd_valid)
wd_test_scaled = scaler_wd.transform(wd_test)

hum_train_scaled = scaler_hum.transform(hum_train)
hum_valid_scaled = scaler_hum.transform(hum_valid)
hum_test_scaled = scaler_hum.transform(hum_test)

vp_train_scaled = scaler_vp.transform(vp_train)
vp_valid_scaled = scaler_vp.transform(vp_valid)
vp_test_scaled = scaler_vp.transform(vp_test)

lp_train_scaled = scaler_lp.transform(lp_train)
lp_valid_scaled = scaler_lp.transform(lp_valid)
lp_test_scaled = scaler_lp.transform(lp_test)

sr_train_scaled = scaler_sr.transform(sr_train)
sr_valid_scaled = scaler_sr.transform(sr_valid)
sr_test_scaled = scaler_sr.transform(sr_test)

month_train_scaled = scaler_month.transform(month_train)
month_valid_scaled = scaler_month.transform(month_valid)
month_test_scaled = scaler_month.transform(month_test)

day_train_scaled = scaler_day.transform(day_train)
day_valid_scaled = scaler_day.transform(day_valid)
day_test_scaled = scaler_day.transform(day_test)

hour_train_scaled = scaler_hour.transform(hour_train)
hour_valid_scaled = scaler_hour.transform(hour_valid)
hour_test_scaled = scaler_hour.transform(hour_test)

data_train = np.hstack((temp_train_scaled, ws_train_scaled, 
                        wd_train_scaled, hum_train_scaled, vp_train_scaled, 
                        lp_train_scaled, sr_train_scaled, month_train_scaled, day_train_scaled, 
                        day_train_scaled))

data_valid = np.hstack((temp_valid_scaled, ws_valid_scaled, 
                        wd_valid_scaled, hum_valid_scaled, vp_valid_scaled, 
                        lp_valid_scaled, sr_valid_scaled,month_valid_scaled, day_valid_scaled, 
                        day_valid_scaled))

data_test = np.hstack((temp_test_scaled, ws_test_scaled, 
                        wd_test_scaled, hum_test_scaled, vp_test_scaled, 
                        lp_test_scaled, sr_test_scaled,month_test_scaled, day_test_scaled, 
                        day_test_scaled))

dataTrainX, dataTrainY = build_dataset(data_train, args.look_back, args.look_ahead)
dataValidX, dataValidY = build_dataset(data_valid, args.look_back, args.look_ahead)
dataTestX, dataTestY = build_dataset(data_test, args.look_back, args.look_ahead)

print("dataTrainX.shape", dataTrainX.shape)
print("dataTrainY.shape", dataTrainY.shape)

dataset_train = Time_Series_Data(dataTrainX, dataTrainY)
dataset_valid = Time_Series_Data(dataValidX, dataValidY)
dataset_test = Time_Series_Data(dataTestX, dataTestY)

dataloader_train = DataLoader(dataset_train, batch_size=args.batch_size, shuffle=True, sampler=None, batch_sampler=None, num_workers=0)
#dataloader_valid = DataLoader(dataset_valid, batch_size=args.batch_size, shuffle=None, sampler=None, batch_sampler=None, num_workers=0)
dataloader_test = DataLoader(dataset_test, batch_size=args.test_batch_size, shuffle=None, sampler=None, batch_sampler=None, num_workers=0)

19536
744
744
dataTrainX.shape (814, 168, 10)
dataTrainY.shape (814, 24, 10)


In [9]:
zeros = scaler_sr.transform(np.zeros((1,1)))
robust_zero = zeros.item()
print(zeros)
print(robust_zero)
print(scaler_sr.inverse_transform(zeros))

[[-0.03529412]]
-0.03529411764705882
[[0.]]


In [10]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, look_ahead):
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.look_ahead = look_ahead
        
        self.relu = nn.ReLU()
        self.mse = nn.MSELoss()
        
        self.lstm = rnn.LSTM(self.input_size, self.hidden_size, batch_first = True)
        self.linear_concat = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, inputs, labels):
        _, lstm_hidden = self.lstm(inputs)
        lstm_hidden_use = lstm_hidden[0].transpose(1,0)
        output = self.linear_concat(lstm_hidden_use)
        #print("output.shape", output.shape)
        #print("labels.shape", labels.shape)
        mse_loss = self.mse(output.squeeze(0), labels)
        return output, mse_loss

In [11]:
hook = sy.TorchHook(torch)
bob_worker = sy.VirtualWorker(hook, id="bob")
alice_worker = sy.VirtualWorker(hook, id="alice")

compute_nodes = [bob_worker, alice_worker]

# create federated train loader
base_dataset = sy.BaseDataset(dataTrainX, dataTrainY)
base_federated = base_dataset.federate(compute_nodes)

federated_train_loader = sy.FederatedDataLoader(
    base_federated,
    batch_size = args.batch_size)

model_workers = {}
model_optims = {}
for worker in compute_nodes:
    model_workers[worker.id] = LSTM(input_size=data_train.shape[1], hidden_size=args.hidden_dim, 
            output_size=args.output_dim, look_ahead=args.look_ahead)
    model_workers[worker.id] = model_workers[worker.id].to(device)
    
    model_optims[worker.id] = optim.Adam(model_workers[worker.id].parameters(), lr=args.learning_rate)

model_fedavg = LSTM(input_size=data_train.shape[1], hidden_size=args.hidden_dim, 
            output_size=args.output_dim, look_ahead=args.look_ahead)
model_fedavg.to(device)

LSTM(
  (relu): ReLU()
  (mse): MSELoss()
  (lstm): LSTM(
    (rnn_forward): ModuleList(
      (0): LSTMCell(
        (fc_xh): Linear(in_features=10, out_features=256, bias=True)
        (fc_hh): Linear(in_features=64, out_features=256, bias=True)
      )
    )
  )
  (linear_concat): Linear(in_features=64, out_features=24, bias=True)
)

In [12]:
def update(data, target, model, optimizer):
    model.send(data.location)
    #print("current model location :",model.location)
    optimizer.zero_grad()
    prediction, loss = model(data, target[:,:,6])
    loss.backward()
    optimizer.step()
    return model.get(), prediction.get(), loss.get()

def train():
    for worker_id, model in model_workers.items():
            model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader):
        loss_got = 0
        data, target = data.to(device), target.to(device)
        model_workers[data.location.id], _, loss = update(data, target, model_workers[data.location.id], model_optims[data.location.id])
        loss_got += loss.item()
        #for worker_id, model in model_workers.items():
        #    model.get()
        model_avg = utils.federated_avg(model_workers)
        for worker_id, model in model_workers.items():
            model = model_avg.copy()
        
        if batch_idx % 8 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBatch loss {:.6f}\t'.format(
                epoch, batch_idx, len(federated_train_loader),
                       100. * batch_idx / len(federated_train_loader), loss_got))
    return model_avg

In [13]:
def test(federated_model):
    federated_model.eval()
    test_loss = 0
    for data, target in dataloader_test:
        data, target = data.to(device), target.to(device)
        prediction, _ = federated_model(data, target[:,:,6])
        test_loss += F.mse_loss(prediction.squeeze(0), target[:,:,6], reduction='sum').item()
        
    test_loss /= len(dataloader_test.dataset)
    print('Test set: Average loss: {:.4f}'.format(test_loss))

In [ ]:
for epoch in range(args.epochs):
    start_time = time.time()
    print(f"Epoch Number {epoch + 1}")
    federated_model = train()
    model_fedavg = federated_model
    test(federated_model)
    total_time = time.time() - start_time
    print('Communication time over the network', round(total_time, 2), 's\n')

Epoch Number 1
Train Epoch: 0 [0/26 (0%)]	Batch loss 1.100080	
Train Epoch: 0 [8/26 (31%)]	Batch loss 2.162728	
Train Epoch: 0 [16/26 (62%)]	Batch loss 0.482679	
Train Epoch: 0 [24/26 (92%)]	Batch loss 0.982881	
Test set: Average loss: 8.4956
Communication time over the network 33.77 s

Epoch Number 2
Train Epoch: 1 [0/26 (0%)]	Batch loss 1.027383	
Train Epoch: 1 [8/26 (31%)]	Batch loss 1.960961	
Train Epoch: 1 [16/26 (62%)]	Batch loss 0.386804	
Train Epoch: 1 [24/26 (92%)]	Batch loss 0.679172	
Test set: Average loss: 5.5829
Communication time over the network 35.64 s

Epoch Number 3
Train Epoch: 2 [0/26 (0%)]	Batch loss 0.641147	
Train Epoch: 2 [8/26 (31%)]	Batch loss 1.261989	
Train Epoch: 2 [16/26 (62%)]	Batch loss 0.159777	
Train Epoch: 2 [24/26 (92%)]	Batch loss 0.220664	
Test set: Average loss: 8.7959
Communication time over the network 35.52 s

Epoch Number 4
Train Epoch: 3 [0/26 (0%)]	Batch loss 0.295444	
Train Epoch: 3 [8/26 (31%)]	Batch loss 0.345731	
Train Epoch: 3 [16/26 (6